In [27]:
import pandas as pd
import yaml 
from yaml import SafeLoader
import config
import openai
import os

In [28]:
openai.api_key = config.api_key

In [60]:
class ChatModel:

    def __init__(self, system: str, dir: str, temperature=0.3):

        """Sets up the model with a system prompt and a temperature"""
        self.model = openai.ChatCompletion()
        self.system = system
        self.temperature = temperature
        self.dir = dir
        self.messages = []

    def reset(self):
        self.messages = [{"role": "system", "content": self.system}]
    
    def few_shot(self, directify: bool):

        """
        Loads the examples from the given directory and adds them to the messages depending on the direction of translation
        dir: directory of the examples
        directify: when set to true, sets up the model to translate from polite->direct
        """

        if directify:
            before, after = "polite", "direct"
        else:
            before, after = "direct", "polite"

        path = os.path.join(os.getcwd(), self.dir)
        for file in os.listdir(path):
            with open(f"{path}/{file}") as f:
                data = yaml.load(f, Loader=SafeLoader)
                # for polite: the prompt is "can you make the following message more {direct}? {polite message}"
                # for direct: the prompt is "can you make the following message more {polite}? {direct message}"
                self.messages.append({"role": "user", "content": f"Can you make the following message more {after}? {data[before]}"})
                self.messages.append({"role": "assistant", "content": f"{data[after]}"})

    def say(self, prompt: str, directify: bool, few_shot: True):
        
        """Prompts the model for a response. This adds the last user prompt to the messages and prompts the model."""

        self.reset()

        if few_shot:
            self.few_shot(directify=directify)
        
        if directify:
            base = "Please rephrase this message to be more direct semi-formal work email, like how a steoreotypical white man with priviledges would write it. Make it more concise."
        else:
            base = "Please make this message more polite, warm and enthusiastic, while still keeping the tone semi-formal.\n"

        self.messages.append({ "role": "user", "content": base + prompt})
        response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=self.messages, temperature=self.temperature)
        return response.choices[0]["message"]["content"]


In [63]:
system_prompt = "You are a translator helping people write better emails."
model = ChatModel(system=system_prompt, dir="examples", temperature=0.2)
model.say("""Hi James, 
My name is Raisa. I am the co-head of the web and software team for the NU Sci magazine. I'm emailing you because I heard from a friend that you're the president of Kaleidoscope and I wanted to learn more about what Kaleidoscope offers for club funding. I was wondering if you had a website or something I could look at by any chance. Thank you so much and have a great rest of your night!""", True, False)

"Dear James,\n\nI'm Raisa, co-head of the web and software team for NU Sci magazine. I heard you're the president of Kaleidoscope and I'd like to know more about club funding. Do you have a website I can check out? Thank you and have a great night!\n\nBest regards,\nRaisa"

In [64]:
model.say("""Hello, can you send over the files I requested earlier? \n Thanks, \nLuisa""", False, False)

'Hi there, \nCould you please send me the files I requested earlier? \nThank you very much, \nLuisa'